In [1]:
import os
import json
import ee
import geemap
import xarray as xr
import wxee
from geemap import geojson_to_ee, ee_to_geojson
from ipyleaflet import GeoJSON

In [2]:
# ee.Authenticate()

In [3]:
ee.Initialize()
wxee.Initialize()

In [4]:
file_path = os.path.abspath("data/resourceshedbb_CostaRica_SanJose.geojson")

with open(file_path) as f:
    json_data = json.load(f)

bbox_geojson = geojson_to_ee(json_data) # need as a feature collection, not bounding box

In [5]:
## set time frame
before_start= '2019-03-01'
before_end='2019-03-10'

after_start='2019-03-10'
after_end='2019-03-23'

In [6]:
# SET SAR PARAMETERS (can be left default)

# Polarization (choose either "VH" or "VV")
polarization = "VH"  # or "VV"

# Pass direction (choose either "DESCENDING" or "ASCENDING")
pass_direction = "DESCENDING"  # or "ASCENDING"

# Difference threshold to be applied on the difference image (after flood - before flood)
# It has been chosen by trial and error. Adjust as needed.
difference_threshold = 1.25

# Relative orbit (optional, if you know the relative orbit for your study area)
# relative_orbit = 79

In [7]:
# Rename the selected geometry feature
aoi = bbox_geojson

# Load and filter Sentinel-1 GRD data by predefined parameters
collection = ee.ImageCollection('COPERNICUS/S1_GRD') \
    .filter(ee.Filter.eq('instrumentMode', 'IW')) \
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', polarization)) \
    .filter(ee.Filter.eq('orbitProperties_pass', pass_direction)) \
    .filter(ee.Filter.eq('resolution_meters', 10)) \
    .filterBounds(aoi) \
    .select(polarization)

# Select images by predefined dates
before_collection = collection.filterDate(before_start, before_end)
after_collection = collection.filterDate(after_start, after_end)

# Create a mosaic of selected tiles and clip to the study area
before = before_collection.mosaic().clip(aoi)
after = after_collection.mosaic().clip(aoi)

# Apply radar speckle reduction by smoothing
smoothing_radius = 50
before_filtered = before.focal_mean(smoothing_radius, 'circle', 'meters')
after_filtered = after.focal_mean(smoothing_radius, 'circle', 'meters')

# Calculate the difference between the before and after images
difference = after_filtered.divide(before_filtered)

# Apply the predefined difference-threshold and create the flood extent mask
threshold = difference_threshold
difference_binary = difference.gt(threshold)

# Refine the flood result using additional datasets
swater = ee.Image('JRC/GSW1_0/GlobalSurfaceWater').select('seasonality')
swater_mask = swater.gte(10).updateMask(swater.gte(10))
flooded_mask = difference_binary.where(swater_mask, 0)
flooded = flooded_mask.updateMask(flooded_mask)
connections = flooded.connectedPixelCount()
flooded = flooded.updateMask(connections.gte(8))

# Mask out areas with more than 5 percent slope using a Digital Elevation Model
DEM = ee.Image('WWF/HydroSHEDS/03VFDEM')
terrain = ee.Algorithms.Terrain(DEM)
slope = terrain.select('slope')
flooded = flooded.updateMask(slope.lt(5))


In [8]:
# Define the Map object
Map = geemap.Map(center=(8.6255, -84.0209), zoom=8)  # Centered at Costa Rica (adjust coordinates as needed)


# Add the flooded areas layer to the map with a blue palette
Map.addLayer(flooded, {'palette': '0000FF'}, 'Flooded areas')

# Display the map
Map


Map(center=[8.6255, -84.0209], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(chi…